<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [8]:
import os
import librosa
import json
import glob
from sklearn.model_selection import train_test_split

# Emotion mapping based on your dataset description
emotion_map = {
    1: "Happy",
    2: "Sad",
    3: "Angry",
    4: "Surprise",
    5: "Neutral"
}

def parse_filename(filename):
    """Parses the filename to extract the emotion label."""
    parts = filename.split('-')
    if len(parts) >= 3:  # Adjust based on your filename structure
        emotion_label = int(parts[2])  # Assumes the 3rd part is the emotion label
        return emotion_map.get(emotion_label, "Unknown")
    return None

def calculate_audio_length(file_path):
    """Calculates the length of an audio file in seconds."""
    audio, sr = librosa.load(file_path, sr=None)
    return librosa.get_duration(y=audio, sr=sr)

def create_filtered_data(directory):
    """Creates a minimal JSON data structure for audio files in a directory, excluding 'Surprise' emotion."""
    json_data = []

    # Using glob to find all WAV files in the directory
    audio_files = glob.glob(os.path.join(directory, '**/*.wav'), recursive=True)

    for file_path in audio_files:
        filename = os.path.basename(file_path)
        audio_id = os.path.splitext(filename)[0]
        emotion_label = parse_filename(filename)

        # Exclude files with emotion "Surprise"
        if emotion_label and emotion_label != "Surprise":
            audio_length = calculate_audio_length(file_path)

            # Create minimal data entry
            data_entry = {
                "id": audio_id,
                "emotion": emotion_label,
                "length": audio_length,
                "wav": file_path
            }
            json_data.append(data_entry)

    return json_data

def split_and_save_data(json_data, seed=32):
    """Splits the data into train and test sets and saves them to separate JSON files."""
    train, test = train_test_split(json_data, test_size=0.2, random_state=seed)

    # Save train data
    with open('train_data.json', 'w') as train_file:
        json.dump(train, train_file, indent=4)

    # Save test data
    with open('test_data.json', 'w') as test_file:
        json.dump(test, test_file, indent=4)



In [9]:
# Directory path for your audio files
directory_path = '/content/t9h6p943xy-5/BanglaSER'

# Step 1: Create filtered data directly
print("Creating filtered data...")
filtered_data = create_filtered_data(directory_path)

# Step 2: Split and save data to train and test JSON files
print("Splitting and saving data into train and test JSON files...")
split_and_save_data(filtered_data)

print("Train and test JSON files created successfully.")


Creating filtered data...
Splitting and saving data into train and test JSON files...
Train and test JSON files created successfully.


In [11]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/t9h6p943xy-5/BanglaSER/train_data.json",'r') as f:
    train_data = json.load(f)

with open("/content/t9h6p943xy-5/BanglaSER/test_data.json",'r') as f:
    test_data = json.load(f)

In [12]:

train_data

[{'id': '03-01-03-02-01-01-02',
  'emotion': 'Angry',
  'length': 3.024489795918367,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-03-02-01-01-02.wav'},
 {'id': '03-01-03-02-01-03-20',
  'emotion': 'Angry',
  'length': 3.4402916666666665,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 20/03-01-03-02-01-03-20.wav'},
 {'id': '03-01-01-01-02-03-21',
  'emotion': 'Happy',
  'length': 3.600793650793651,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 21/03-01-01-01-02-03-21.wav'},
 {'id': '03-01-01-01-03-01-28',
  'emotion': 'Happy',
  'length': 3.0531041666666665,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 28/03-01-01-01-03-01-28.wav'},
 {'id': '03-01-03-02-03-03-24',
  'emotion': 'Angry',
  'length': 3.0224166666666665,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 24/03-01-03-02-03-03-24.wav'},
 {'id': '03-01-03-02-03-02-24',
  'emotion': 'Angry',
  'length': 3.1719583333333334,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 24/03-01-03-02-03-02-24.wav'},
 {'id': '03-

In [13]:
test_data

[{'id': '03-01-03-02-01-02-07',
  'emotion': 'Angry',
  'length': 3.026125,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 07/03-01-03-02-01-02-07.wav'},
 {'id': '03-01-03-02-01-02-32',
  'emotion': 'Angry',
  'length': 3.0920181405895693,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 32/03-01-03-02-01-02-32.wav'},
 {'id': '03-01-03-02-01-01-11',
  'emotion': 'Angry',
  'length': 3.0717913832199546,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 11/03-01-03-02-01-01-11.wav'},
 {'id': '03-01-05-01-03-03-22',
  'emotion': 'Neutral',
  'length': 3.4072562358276643,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 22/03-01-05-01-03-03-22.wav'},
 {'id': '03-01-02-01-03-03-32',
  'emotion': 'Sad',
  'length': 3.35421768707483,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 32/03-01-02-01-03-03-32.wav'},
 {'id': '03-01-02-01-03-03-12',
  'emotion': 'Sad',
  'length': 3.3862083333333333,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 12/03-01-02-01-03-03-12.wav'},
 {'id': '03-01-02-01-03-

In [15]:

train_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(train_data)
]
train_dataset = Dataset.from_list(train_samples_list)

In [17]:
test_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(test_data)
]
test_dataset = Dataset.from_list(test_samples_list)

In [18]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
})

In [36]:
def add_base_path(example):
    # Check if base path is already in the file path
    if not example['wav'].startswith('/content/t9h6p943xy-5/BanglaSER/'):
        example['wav'] = '/content/t9h6p943xy-5/BanglaSER/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

In [37]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [38]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/t9h6p943xy-5/BanglaSER//content/t9h6p943xy-5//content//content/t9h6p943xy-5//content/t9h6p943xy-5/BanglaSER//content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-03-02-01-01-02.wav'